# Avance 4: Modelos Alternativos

**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC5035.10 - Proyecto Integrador**

Tecnológico de Monterrey

**Profesor:** Dr. Horacio Martínez Alfaro

---

**Equipo #:** [Tu Número de Equipo Aquí]

**Integrantes:**
* [Nombre Completo - Matrícula]
* [Nombre Completo - Matrícula]
* [Nombre Completo - Matrícula]

---

## Objetivo

Este avance implica construir múltiples modelos (individuales, no ensambles) relevantes para resolver el problema de clasificación de gestos y evaluar su desempeño. Diferentes algoritmos pueden comportarse de manera óptima en diferentes tipos de datos o tareas. La construcción de modelos alternativos permite explorar y evaluar cuál de ellos proporciona el mejor rendimiento para este problema particular.

Además, los modelos se pueden ajustar para determinar si se puede mejorar su rendimiento. Diferentes configuraciones de hiperparámetros pueden afectar significativamente el rendimiento de un modelo. Construir modelos alternativos implica explorar y ajustar estos hiperparámetros para encontrar la configuración óptima.

Las siguientes son acciones que se abordarán en este avance:
1.  Construir al menos 6 modelos diferentes (individuales, no ensambles), utilizando algoritmos variados.
2.  Comparar el rendimiento de los modelos obtenidos.
3.  Seleccionar los dos modelos que proporcionen el mejor rendimiento.
4.  Ajustar los dos mejores modelos.
5.  Elegir el modelo individual final.

## 1. Configuración del Entorno e Importación de Librerías

In [ ]:
# Instalación de gdown para descargar el dataset desde Google Drive (si no está instalado)
!pip install -q gdown

# Librerías generales
import pandas as pd
import numpy as np
import time
import os
import gdown

# Preprocesamiento y división de datos
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier

# Métricas de evaluación
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Ignorar advertencias (opcional, para limpieza de salida)
import warnings
warnings.filterwarnings('ignore')

## 2. Carga y Preparación de Datos

Se cargará el conjunto de datos `gestures_balanced.csv`. Este archivo contiene características (componentes principales) y etiquetas para la clasificación de gestos.

In [ ]:
# ID del archivo en Google Drive
file_id = '1KsgEyYatG6dSkEJwjNyva43IBDoyW4li'
output_path = 'gestures_balanced.csv'

# Descargar el archivo si no existe localmente
if not os.path.exists(output_path):
    print(f"Descargando {output_path}...")
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)
else:
    print(f"{output_path} ya existe.")

# Cargar el dataset
df = pd.read_csv(output_path)

print("\nPrimeras 5 filas del dataset:")
print(df.head())

print("\nInformación del dataset:")
df.info()

print("\nDistribución de clases:")
print(df['label'].value_counts())

### 2.1. Separación de Características (X) y Variable Objetivo (y)

In [ ]:
X = df.drop('label', axis=1)
y_raw = df['label']

# Codificar las etiquetas (variable objetivo) a números, ya que algunos modelos lo requieren
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_raw)
class_names = label_encoder.classes_  # Guardar los nombres originales de las clases

print("Características (X) shape:", X.shape)
print("Variable objetivo (y) shape:", y.shape)
print("Primeras 5 etiquetas codificadas:", y[:5])
print("Nombres de las clases originales:", class_names)

### 2.2. División en Conjuntos de Entrenamiento y Prueba
Se dividirá el dataset en 80% para entrenamiento y 20% para prueba, utilizando estratificación para mantener la proporción de clases en ambos conjuntos.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Tamaño de X_train:", X_train.shape)
print("Tamaño de X_test:", X_test.shape)
print("Tamaño de y_train:", y_train.shape)
print("Tamaño de y_test:", y_test.shape)

### 2.3. Escalado de Características (Opcional pero Recomendado)
Dado que los datos ya son componentes principales (resultado de PCA), podrían estar ya escalados. Sin embargo, para algoritmos sensibles a la escala como SVM o KNN, aplicar `StandardScaler` puede ser beneficioso.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

print("Primeras 5 filas de X_train escalado (primeras 5 características):")
print(X_train_scaled[:5, :5])

## 3. Construcción y Evaluación de Modelos Individuales

Se construirán y evaluarán al menos 6 modelos de clasificación individuales. Para cada modelo, se medirá:
- Tiempo de entrenamiento.
- Accuracy.
- F1-Score (Macro).
- Precision (Macro).
- Recall (Macro).

La métrica principal para la comparación será el **F1-Score (Macro)**.

In [ ]:
models = {
    "Regresión Logística": LogisticRegression(max_iter=1000, random_state=42, solver='liblinear'),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(n_neighbors=5),
    "Árbol de Decisión": DecisionTreeClassifier(random_state=42),
    "SVM (Linear)": SVC(kernel='linear', probability=True, random_state=42),
    "SVM (RBF)": SVC(kernel='rbf', probability=True, random_state=42),
    "Gaussian NB": GaussianNB(),
    "LDA": LinearDiscriminantAnalysis(),
    "MLP": MLPClassifier(max_iter=500, random_state=42, hidden_layer_sizes=(64,), early_stopping=True)
}

results = []
for name, m in models.items():
    print(f"Entrenando {name}…")
    start = time.time()
    if name in ["Árbol de Decisión", "Gaussian NB"]:
        m.fit(X_train, y_train)
        pred = m.predict(X_test)
    else:
        m.fit(X_train_scaled, y_train)
        pred = m.predict(X_test_scaled)
    t = time.time() - start
    results.append({
        "Modelo": name,
        "Tiempo (s)": round(t,4),
        "Accuracy": round(accuracy_score(y_test,pred),4),
        "F1-Macro": round(f1_score(y_test,pred,average='macro'),4)
    })
df_results = pd.DataFrame(results).sort_values("F1-Macro",ascending=False)
df_results

## 4. Selección de los Dos Mejores Modelos y Ajuste de Hiperparámetros
– Continúa con GridSearchCV sobre los dos primeros…

## 7. Conclusiones del Avance

**[Aquí resume tus conclusiones principales.]**